## Super Resolution

In [1]:
# from PIL import Image
# import PIL
# import cv2
# import time

# img = cv2.imread('../temp/corrected.jpg')
# width = img.shape[1]
# height = img.shape[0]
# bicubic = cv2.resize(img,(width*4,height*4))
# # cv2.imshow('Image',img)
# # cv2.imshow('BICUBIC',bicubic)

# super_res = cv2.dnn_superres.DnnSuperResImpl_create()

# # start = time.time()
# # super_res.readModel('LapSRN_x4.pb')
# # super_res.setModel('lapsrn',4)
# # lapsrn_image = super_res.upsample(img)
# # end = time.time()
# # print('Time taken in seconds by lapsrn', end-start)
# # cv2.imshow('LAPSRN',lapsrn_image)

# start = time.time()
# super_res.readModel('EDSR_x4.pb')
# super_res.setModel('edsr',4)
# edsr_image = super_res.upsample(img)
# end = time.time()
# print('Time taken in seconds by edsr', end-start)
# # cv2.imshow('EDSR',edsr_image)

# cv2.imwrite("../temp/upscaled.jpg", edsr_image)

# # cv2.waitKey(0)
# # cv2.destroyAllWindows()


## Perspective Correction

In [2]:
import cv2

from cvtools import resize
from cvtools import perspective_transform
from cvtools import getoutlines
from cvtools import simple_erode
from cvtools import simple_dilate
from cvtools import brightness_contrast
from cvtools import blank

# READ INPUT IMAGE
img = cv2.imread("../temp/image.jpg")

try:
    # if input image is empty, notify the user and quit program
    if img is None:
        print()
        print("The file does not exist or is empty!")
        print("Please select a valid image file!")
        print()
        exit(0)  # exit code zero means a clean exit with no output/errors etc.


    """
    Primary Functions
    """


    def preprocess(img):
        """
        BAISC PRE-PROCESSING TO OBTAIN A CANNY EDGE IMAGE
        """

        # increase contrast between paper and background
        img_adj = brightness_contrast(img, 1.56, -60)

        # calculate the ratio of the image to the new height (500px) so we
        # can scale the manipulated image back to the original size later
        scale = img_adj.shape[0] / 500.0

        # scale the image down to 500px in height;
        img_scaled = resize(img_adj, height=500)

        # convert image to grayscale
        img_gray = cv2.cvtColor(img_scaled, cv2.COLOR_BGR2GRAY)

        # apply gaussian blur with a 11x11 kernel
        img_gray = cv2.GaussianBlur(img_gray, (11, 11), 0)

        # apply canny edge detection
        img_edge = cv2.Canny(img_gray, 60, 245)

        # dilate the edge image to connect any small gaps
        img_edge = simple_dilate(img_edge)

        return img_adj, scale, img_scaled, img_edge


    def gethull(img_edge):
        """
        1st ROUND OF OUTLINE FINDING, + CONVEX HULL
        """

        # make a copy of the edge image because the following function manipulates
        # the input
        img_prehull = img_edge.copy()

        # find outlines in the (newly copied) edge image
        outlines = getoutlines(img_prehull)

        # create a blank image for convex hull operation
        img_hull = blank(img_prehull.shape, img_prehull.dtype, "0")

        # draw convex hulls (fit polygon) for all outlines detected to 'img_contour'
        for outline in range(len(outlines)):

            hull = cv2.convexHull(outlines[outline])

            # parameters: source image, outlines (contours),
            #             contour index (-1 for all), color, thickness
            cv2.drawContours(img_hull, [hull], 0, 255, 3)

        # erode the hull image to make the outline closer to paper
        img_hull = simple_erode(img_hull)

        return img_hull


    def getcorners(img_hull):
        """
        2nd ROUND OF OUTLINE FINDING, + SORTING & APPROXIMATION
        """

        # make a copy of the edge image because the following function manipulates
        # the input
        img_outlines = img_hull.copy()

        # find outlines in the convex hull image
        outlines = getoutlines(img_outlines)

        # sort the outlines by area from large to small, and only take the largest 4
        # outlines in order to speed up the process and not waste time
        outlines = sorted(outlines, key=cv2.contourArea, reverse=True)[:4]

        # loop over outlines
        for outline in outlines:

            # find the perimeter of each outline for use in approximation
            perimeter = cv2.arcLength(outline, True)

            # > approximate a rough contour for each outline found, with (hopefully)
            #   4 points (rectangular sheet of paper); [Douglas-Peuker Algorithm]
            # > FIRST OPTION is the input outline;
            # > SECOND OPTION is the accuracy of approximation (epsilon), here it
            #   is set to a percentage of the perimeter of the outline
            # > THIRD OPTION is whether to assume an outline
            #   is closed, which in this case is yes (sheet of paper)
            approx = cv2.approxPolyDP(outline, 0.02 * perimeter, True)

            # if the approximation has 4 points, then assume it is correct, and
            # assign these points to the 'corners' variable
            if len(approx) == 4:
                corners = approx
                break

        return corners


    """
    Main Proccess of the Program
    """

    # obtain the adjusted image, scaled image along with its scale factor, and the
    # Canny edge image
    img_adj, scale, img_scaled, img_edge = preprocess(img)

    # perform convex hull on edge image to prevent imcomplete outline
    img_hull = gethull(img_edge)

    # obtain 4 corner points of the convex hull image
    corners = getcorners(img_hull)

    # scale the corner points back to the original size of the image using the scale
    # calculated previously
    corners = corners.reshape(4, 2) * scale

    # finally correct the perspective of the image by applying four-point
    # perspective transform
    img_corrected = perspective_transform(img_adj, corners)

    # write corrected image to file
    cv2.imwrite("../temp/corrected.jpg", img_corrected)
except:
    cv2.imwrite("../temp/corrected.jpg", img)

## Perspective Correction

In [3]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# def get_destination_points(corners):
#     """
#     -Get destination points from corners of warped images
#     -Approximating height and width of the rectangle: we take maximum of the 2 widths and 2 heights
#     Args:
#         corners: list
#     Returns:
#         destination_corners: list
#         height: int
#         width: int
#     """

#     w1 = np.sqrt((corners[0][0] - corners[1][0]) ** 2 + (corners[0][1] - corners[1][1]) ** 2)
#     w2 = np.sqrt((corners[2][0] - corners[3][0]) ** 2 + (corners[2][1] - corners[3][1]) ** 2)
#     w = max(int(w1), int(w2))

#     h1 = np.sqrt((corners[0][0] - corners[2][0]) ** 2 + (corners[0][1] - corners[2][1]) ** 2)
#     h2 = np.sqrt((corners[1][0] - corners[3][0]) ** 2 + (corners[1][1] - corners[3][1]) ** 2)
#     h = max(int(h1), int(h2))

#     destination_corners = np.float32([(0, 0), (w - 1, 0), (0, h - 1), (w - 1, h - 1)])

# #     print('\nThe destination points are: \n')
#     for index, c in enumerate(destination_corners):
#         character = chr(65 + index) + "'"
# #         print(character, ':', c)

# #     print('\nThe approximated height and width of the original image is: \n', (h, w))
#     return destination_corners, h, w

# def unwarp(img, src, dst):
#     """
#     Args:
#         img: np.array
#         src: list
#         dst: list
#     Returns:
#         un_warped: np.array
#     """
#     h, w = img.shape[:2]
#     H, _ = cv2.findHomography(src, dst, method=cv2.RANSAC, ransacReprojThreshold=3.0)
# #     print('\nThe homography matrix is: \n', H)
#     un_warped = cv2.warpPerspective(img, H, (w, h), flags=cv2.INTER_LINEAR)

#     # plot

# #     f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
#     # f.subplots_adjust(hspace=.2, wspace=.05)
# #     ax1.imshow(img)
# #     ax1.set_title('Original Image')

#     x = [src[0][0], src[2][0], src[3][0], src[1][0], src[0][0]]
#     y = [src[0][1], src[2][1], src[3][1], src[1][1], src[0][1]]

# #     ax2.imshow(img)
# #     ax2.plot(x, y, color='yellow', linewidth=3)
# #     ax2.set_ylim([h, 0])
# #     ax2.set_xlim([0, w])
# #     ax2.set_title('Target Area')

# #     plt.show()
#     return un_warped

# def apply_filter(image):
#     """
#     Define a 5X5 kernel and apply the filter to gray scale image
#     Args:
#         image: np.array
#     Returns:
#         filtered: np.array
#     """
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     kernel = np.ones((5, 5), np.float32) / 15
#     filtered = cv2.filter2D(gray, -1, kernel)
# #     plt.imshow(cv2.cvtColor(filtered, cv2.COLOR_BGR2RGB))
# #     plt.title('Filtered Image')
# #     plt.show()
#     return filtered

# def apply_threshold(filtered):
#     """
#     Apply OTSU threshold
#     Args:
#         filtered: np.array
#     Returns:
#         thresh: np.array
#     """
#     ret, thresh = cv2.threshold(filtered, 250, 255, cv2.THRESH_OTSU)
# #     plt.imshow(cv2.cvtColor(thresh, cv2.COLOR_BGR2RGB))
# #     plt.title('After applying OTSU threshold')
# #     plt.show()
#     return thresh

# def detect_contour(img, image_shape):
#     """
#     Args:
#         img: np.array()
#         image_shape: tuple
#     Returns:
#         canvas: np.array()
#         cnt: list
#     """
#     canvas = np.zeros(image_shape, np.uint8)
#     contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
#     cnt = sorted(contours, key=cv2.contourArea, reverse=True)[0]
#     cv2.drawContours(canvas, cnt, -1, (0, 255, 255), 3)
# #     plt.title('Largest Contour')
# #     plt.imshow(canvas)
# #     plt.show()

#     return canvas, cnt

# def detect_corners_from_contour(canvas, cnt):
#     """
#     Detecting corner points form contours using cv2.approxPolyDP()
#     Args:
#         canvas: np.array()
#         cnt: list
#     Returns:
#         approx_corners: list
#     """
#     epsilon = 0.02 * cv2.arcLength(cnt, True)
#     approx_corners = cv2.approxPolyDP(cnt, epsilon, True)
#     cv2.drawContours(canvas, approx_corners, -1, (255, 255, 0), 10)
#     approx_corners = sorted(np.concatenate(approx_corners).tolist())
# #     print('\nThe corner points are ...\n')
#     for index, c in enumerate(approx_corners):
#         character = chr(65 + index)
# #         print(character, ':', c)
#         cv2.putText(canvas, character, tuple(c), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

#     # Rearranging the order of the corner points
#     approx_corners = [approx_corners[i] for i in [0, 2, 1, 3]]

# #     plt.imshow(canvas)
# #     plt.title('Corner Points: Douglas-Peucker')
# #     plt.show()
#     return approx_corners

# def example_two():
#     """
#     Skew correction using homography and corner detection using contour points
#     Returns: None
#     """
#     image = cv2.imread('../temp/image.jpg')
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# #     plt.imshow(image)
# #     plt.title('Original Image')
# #     plt.show()

#     filtered_image = apply_filter(image)
#     threshold_image = apply_threshold(filtered_image)

#     cnv, largest_contour = detect_contour(threshold_image, image.shape)
#     corners = detect_corners_from_contour(cnv, largest_contour)

#     destination_points, h, w = get_destination_points(corners)
#     un_warped = unwarp(image, np.float32(corners), destination_points)

#     cropped = un_warped[0:h, 0:w]
# #     f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
#     # f.subplots_adjust(hspace=.2, wspace=.05)
# #     ax1.imshow(un_warped)
# #     ax2.imshow(cropped)
#     cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
#     cv2.imwrite("../temp/cropped.jpg",cropped)

# #     plt.show()

# example_two()

## 00: Opening an Image

In [4]:
import cv2
from matplotlib import pyplot as plt
image_file = "../temp/corrected.jpg"
img = cv2.imread(image_file)

In [5]:
# def display(im_path):
#     dpi = 80
#     im_data = plt.imread(im_path)

#     height, width  = im_data.shape[:2]
    
#     # What size does the figure need to be in inches to fit the image?
#     figsize = width / float(dpi), height / float(dpi)

#     # Create a figure of the right size with one axes that takes up the full figure
#     fig = plt.figure(figsize=figsize)
#     ax = fig.add_axes([0, 0, 1, 1])

#     # Hide spines, ticks, etc.
#     ax.axis('off')

#     # Display the image.
#     ax.imshow(im_data, cmap='gray')

#     plt.show()

In [6]:
# display(image_file)

## 01: Deskew

In [7]:
import numpy as np
from scipy.ndimage import interpolation as inter

def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1, dtype=float)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2, dtype=float)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    corrected = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
            borderMode=cv2.BORDER_REPLICATE)

    return best_angle, corrected

angle, corrected = correct_skew(img)

C:\Users\aksha\AppData\Local\Temp\ipykernel_1236\2806705659.py:6: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  data = inter.rotate(arr, angle, reshape=False, order=0)


In [8]:
cv2.imwrite("../temp/result1.jpg", corrected)

True

## 02: Inverted Images

In [9]:
inverted_image = cv2.bitwise_not(corrected)

In [10]:
cv2.imwrite("../temp/result2.jpg", inverted_image)

True

## 03: Binarization

In [11]:
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

gray_image = grayscale(inverted_image)

thresh, im_bw = cv2.threshold(gray_image, 210, 230, cv2.THRESH_BINARY)

In [12]:
cv2.imwrite("../temp/result3.jpg", im_bw)

True

## 04: Noise Removal

In [13]:
def noise_removal(image):
    import numpy as np
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return (image)

no_noise = noise_removal(im_bw)

In [14]:
cv2.imwrite("../temp/result4.jpg", no_noise)

True

## 05: Dilation and Erosion

In [15]:
def thin_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2),np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)

In [16]:
eroded_image = thin_font(no_noise)
cv2.imwrite("../temp/result5.jpg", eroded_image)

True

In [17]:
def thick_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2),np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)

In [18]:
dilated_image = thick_font(no_noise)
cv2.imwrite("../temp/result6.jpg", dilated_image)

True